# Lab2 Moment tensor inversion

In this lab you will learn:
- **Download and process data.**
- **Calculate Green's functions.**
- **Calculate moment tensor using tdmtpy.**

## An example earthquake 2019-10-15 05:33:42 (UTC) 1km SSE of Pleasant Hill, CA
[USGS event information for the earthquake](https://earthquake.usgs.gov/earthquakes/eventpage/nc73291880/executive)


## Package/module Requirements 
Green's functions are computed using the software package Computer Porgrams in Seismology (CPS) by Robert Herrmann (http://www.eas.slu.edu/eqc/eqccps.html). On `compute2`, we have installed the CPS for you.

Also, you will the packages for Python3: `obsPy`, `pandas`, `matplotlib`, `numpy`, `mttime`. We have pre-installed them for you. Or you can install it by youself by using the command `pip3` in your Linux terminal:

```bash
pip3 install --user obspy pandas matplotlib numpy 
pip3 install --user  git+https://github.com/LLNL/mttime
```




# 1. Download data


In [1]:
# Import third-party libraries
from pathlib import Path
from obspy.clients.fdsn import Client
from obspy import read_events, UTCDateTime
from obspy.clients.fdsn.mass_downloader import CircularDomain, Restrictions, MassDownloader

In [2]:
# 1.1 lets search for and download information about an earthquake
#     The information will be used for downloading waveforms in #1.2.

event_bool = True
if event_bool:
    dataCenter="IRIS"
    client = Client(dataCenter)
    starttime = UTCDateTime("2019-10-15T00:00:00")
    endtime = UTCDateTime("2019-10-16T23:59:59")
    catalog = client.get_events(starttime=starttime, endtime=endtime,
                        minmagnitude=4,maxmagnitude=5,
                        minlatitude=36, maxlatitude=38,
                        minlongitude=-123, maxlongitude=-122) # we set several search options
    catalog.write("quakes.xml",format="QUAKEML")


In [3]:
# 1.2 Download the waveforms and station metadata 
#     from the Northern California Earthquake Data Center (NCEDC)
#     using ObsPy's mass_downloader function.

dataCenter="NCEDC" 

# Set the time window for retrieving seismograms
time_before = 60 # Time before event origin
time_after = 300 # Time after event origin
download_bool = True

catalog = read_events("quakes.xml")  # the `quakes.xml` was obtained in #1.1
for event in catalog:
    # Create a folder for the event
    evid = str(catalog[0].origins[0].resource_id).split("=")[-1] # User origin resource id as the event id
    outdir = evid
    Path(outdir).mkdir(parents=True,exist_ok=True)
    
    # Then, we will search for seismograms by providing information of the event
    ## the event origin time
    origin_time = event.preferred_origin().time
    starttime = origin_time - time_before
    endtime = origin_time + time_after
    
    ## the event location
    evlo = event.preferred_origin().longitude
    evla = event.preferred_origin().latitude
    depth = event.preferred_origin().depth # in meters
    
    ## set the search area
    domain = CircularDomain(latitude=evla, longitude=evlo, minradius=0.7, maxradius=1.3)
    
    ## set the search period and additional criteria
    restrictions = Restrictions(starttime=starttime, endtime=endtime,
        reject_channels_with_gaps=True, minimum_length=0.95, network="BK",
        channel_priorities=["BH[ZNE12]", "HH[ZNE12]"],
        sanitize=True)
    
    ## save catalog info to file
    event_out = (
        "{evid:s},{origin:s},{jdate:s},"
        "{lon:.4f},{lat:.4f},{depth:.4f},"
        "{mag:.2f},{auth:s}\n"
        )        
    if event.preferred_magnitude() is None:
        mag = -999.
        magtype = "ml"
    else:
        mag = event.preferred_magnitude().mag
        magtype = event.preferred_magnitude().magnitude_type.lower()
    if event.preferred_origin().extra.catalog.value is None:
        auth = "unknown"
    else:
        auth = event.preferred_origin().extra.catalog.value.replace(" ","")
        
    event_out = event_out.format(
        evid=evid,
        origin=str(origin_time),
        jdate="%s%s"%(origin_time.year,origin_time.julday),
        lon=evlo,
        lat=evla,
        depth=depth/1E3,
        mag=mag,
        auth=auth
        )
        
    outfile = "datetime.csv" # we will catalog info to this file
    with open(outfile,"w") as f:
        f.write("evid,origin,jdate,lon,lat,depth,%s,auth\n"%magtype)
        f.write(event_out)
        
    ## Finally, Dowanload waveforms and metadata
    if download_bool:
        mseed_storage = "%s/waveforms"%outdir
        stationxml_storage = "%s/stations"%outdir
        mdl = MassDownloader(providers=[dataCenter])
        mdl_helper = mdl.download(domain, restrictions,
            mseed_storage=mseed_storage,stationxml_storage=stationxml_storage)
        print("Download completed into directotry %s" % (outdir) )
        
        
    print("%s is DONE."%outdir)

[2021-08-24 22:01:01,923] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for NCEDC.
[2021-08-24 22:01:02,491] - obspy.clients.fdsn.mass_downloader - INFO: Successfully initialized 1 client(s): NCEDC.
[2021-08-24 22:01:02,492] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired or preexisting stations: 0
[2021-08-24 22:01:02,493] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Requesting unreliable availability.
[2021-08-24 22:01:05,396] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Successfully requested availability (2.90 seconds)
[2021-08-24 22:01:05,457] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Found 7 stations (21 channels).
[2021-08-24 22:01:05,458] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Will attempt to download data from 7 stations.
[2021-08-24 22:01:05,462] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Status for 21 time intervals/channels before downloadin

Download completed into directotry 41423391
41423391 is DONE.


# **Now we've downloaded the raw data, the next step is to process them.**